### Closures

Давайте рассмотрим концепцию ячейки (cell) для создания косвенной ссылки для переменных, находящихся в нескольких областях действия.

In [1]:
def outer():
    x = 'python'
    def inner():
        print(x, hex(id(x)))
    return inner

In [2]:
fn = outer()
fn()

python 0x7f09877a76c0


In [3]:
fn.__code__.co_freevars

('x',)

Как мы видим, `x` является свободной переменной в замыкании.

In [10]:
fn.__closure__

(<cell at 0x7f47a5c7efe0: str object at 0x7f47b97a76c0>,)

Здесь мы видим, что свободная переменная `x` на самом деле является ссылкой на объект ячейки, который сам по себе является ссылкой на строковый объект.

Давайте посмотрим, какой адрес памяти `x` находится во внешней функции и во внутренней функции. Чтобы убедиться, что интернирование строк не играет роли, я собираюсь использовать объект, который, как мы знаем, Python не будет автоматически интернировать, например список.

In [3]:
def outer():
    x = [1, 2, 3]
    print('outer:', hex(id(x)))
    def inner():
        print('inner:', hex(id(x)))
        print(x)
    return inner

In [4]:
fn = outer()

outer: 0x7f0963ccdd80


In [5]:
fn.__closure__

(<cell at 0x7f0963cbf190: list object at 0x7f0963ccdd80>,)

In [6]:
fn()

inner: 0x7f0963ccdd80
[1, 2, 3]


Как вы можете видеть, каждый адрес памяти `x` в `outer`, `inner` и `ячейке` указывает на один и тот же объект.

#### Modifying the Free Variable

Мы знаем, что можем изменять нелокальные переменные, используя ключевое слово `nonlocal`. Так что следующее будет работать:

In [7]:
def counter():
    count = 0 # local variable

    def inc():
        nonlocal count  # this is the count variable in counter
        count += 1
        return count
    return inc

In [8]:
c = counter()

In [9]:
c()

1

In [10]:
c()

2

##### Shared Extended Scopes

Мы можем устанавливать нелокальные переменные в различных внутренних функциях, которые ссылаются на одну и ту же внешнюю переменную области видимости, т. е. у нас есть свободная переменная, которая является общей для двух замыканий. Это работает, потому что и нелокальные переменные, и внешняя локальная переменная указывают на один и тот же объект ячейки.

In [13]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count += 1
        return count

    def inc2():
        nonlocal count
        count += 1
        return count

    return inc1, inc2

In [14]:
fn1, fn2 = outer()

In [15]:
fn1.__closure__, fn2.__closure__

((<cell at 0x0000015F5299B738: int object at 0x00000000506FEC50>,),
 (<cell at 0x0000015F5299B738: int object at 0x00000000506FEC50>,))

Как вы можете видеть, метка `count` указывает на ту же ячейку.

In [16]:
fn1()

1

In [17]:
fn1()

2

In [18]:
fn2()

3

### Multiple Instances of Closures

Напомним, что **каждый** раз, когда вызывается функция, создается **новая** локальная область действия.

In [19]:
from time import perf_counter

def func():
    x = perf_counter()
    print(x, id(x))

In [20]:
func()

2.7089464582150425e-07 1508916709680


In [21]:
func()

0.011222623387093279 1508916709680


То же самое происходит и с замыканиями, у них есть своя расширенная область действия каждый раз, когда создается замыкание:

In [22]:
def pow(n):
    # n is local to pow
    def inner(x):
        # x is local to inner
        return x ** n
    return inner

В этом примере `n` в функции `inner` является свободной переменной, поэтому у нас есть замыкание, которое содержит `inner` и свободную переменную `n`.

In [23]:
square = pow(2)

In [24]:
square(5)

25

In [25]:
cube = pow(3)

In [26]:
cube(5)

125

Мы видим, что ячейка, используемая для свободной переменной в обоих случаях, **разная**:

In [27]:
square.__closure__

(<cell at 0x0000015F5299B8B8: int object at 0x00000000506FEC90>,)

In [28]:
cube.__closure__

(<cell at 0x0000015F5299BAC8: int object at 0x00000000506FECB0>,)

На самом деле, эти функции (`квадрат` и `куб`) **не** являются одними и теми же функциями, хотя они были «созданы» из одной и той же функции `степени`:

In [29]:
id(square), id(cube)

(1508919294560, 1508919295784)

### Beware (Остерегаться)!

Помните, я говорил, что захваченная переменная — это ссылка, устанавливаемая при создании замыкания, но значение ищется только после вызова функции?

Это может привести к появлению едва заметных ошибок в вашей программе.

Рассмотрим следующий пример, в котором мы хотим создать несколько функций, которые могут прибавлять 1, 2, 3, 4 и к любому переданному им числу.

Мы могли бы сделать следующее:

In [30]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [31]:
add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)
add_4 = adder(4)

In [32]:
add_1(10), add_2(10), add_3(10), add_4(10)

(11, 12, 13, 14)

Но предположим, что мы хотим пойти немного дальше и сделать это следующим образом:

In [33]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x: x + n)
    return adders

In [34]:
adders = create_adders()

Технически в списке `adders` у нас есть 4 функции:

In [35]:
adders

[<function __main__.create_adders.<locals>.<lambda>>,
 <function __main__.create_adders.<locals>.<lambda>>,
 <function __main__.create_adders.<locals>.<lambda>>,
 <function __main__.create_adders.<locals>.<lambda>>]

Первый должен добавить 1 к значению, которое мы ему передаем, второй — 2 и так далее.

In [36]:
adders[3](10)

14

Да, это работает для 4-й функции.

In [37]:
adders[0](10)

14

Ой-ой - что случилось? На самом деле мы получаем одинаковое поведение от каждой из этих функций:

In [38]:
adders[0](10), adders[1](10), adders[2](10), adders[3](10)

(14, 14, 14, 14)

Помните, что я говорил о том, когда захватывается переменная и когда ищется ее значение?

Когда лямбды **создаются**, их `n` - это `n`, используемый в цикле - **то же самое** `n`!!

In [39]:
adders[0].__code__.co_freevars

('n',)

In [40]:
adders[0].__closure__

(<cell at 0x0000015F5299B3D8: int object at 0x00000000506FECD0>,)

In [41]:
adders[1].__closure__

(<cell at 0x0000015F5299B3D8: int object at 0x00000000506FECD0>,)

In [42]:
adders[2].__closure__

(<cell at 0x0000015F5299B3D8: int object at 0x00000000506FECD0>,)

In [43]:
adders[3].__closure__

(<cell at 0x0000015F5299B3D8: int object at 0x00000000506FECD0>,)

Таким образом, к моменту вызова `adder[in]` свободная переменная `in` (общая для всех сумматоров) устанавливается равной 4.

In [44]:
hex(id(4))

'0x506fecd0'

Как мы видим, адрес памяти синглтона — это целое число 4, на которое указывает эта ячейка.

Если вы хотите использовать цикл, чтобы сделать это, и не использовать одну и ту же ячейку для каждой из свободных переменных, мы можем использовать простой трюк, который принудительно вычисляет `n` во время **создания** замыкания, а не при вычислении функции замыкания.

Мы можем сделать это, создав параметр для `n` в нашей лямбде, значением по умолчанию которого является текущее значение `n` —  значения параметров по умолчанию вычисляются при создании функции, а не при ее вызове.

In [45]:
def create_adders():
    adders = []
    for n in range(1, 5):
        adders.append(lambda x, step=n: x + step)
    return adders

In [46]:
adders = create_adders()

In [47]:
adders[0].__closure__

Почему мы ничего не получаем в замыкании? Что насчет свободных переменных?

In [48]:
adders[0].__code__.co_freevars

()

Хм, тоже ничего... Почему?

Ну, посмотрите на лямбду в этом цикле. Ссылается ли она на переменную `n` (кроме значения по умолчанию)? Нет. Следовательно, `n` **не** является свободной переменной в этом случае, и наша лямбда — это просто лямбда, а не замыкание.

И этот код теперь будет работать так, как и ожидалось:

In [49]:
adders[0](10)

11

In [50]:
adders[1](10)

12

In [51]:
adders[2](10)

13

In [52]:
adders[3](10)

14

Вам просто нужно понять, что поскольку значения по умолчанию вычисляются при **создании** функции (в данном случае лямбды), то текущее значение `n` присваивается локальной переменной `step`. Поэтому `step` не будет меняться каждый раз при вызове лямбды, а поскольку на `n` нет ссылки внутри функции (и, следовательно, она вычисляется при вызове лямбды), `n` не является свободной переменной.

#### Nested Closures

Мы также можем вкладывать замыкания, как показано в этом примере:

In [53]:
def incrementer(n):
    def inner(start):
        current = start
        def inc():
            a = 10  # local var
            nonlocal current
            current += n
            return current
        return inc
    return inner


In [54]:
fn = incrementer(2)

In [55]:
fn

<function __main__.incrementer.<locals>.inner>

In [56]:
fn.__code__.co_freevars

('n',)

In [57]:
fn.__closure__

(<cell at 0x0000015F5299B798: int object at 0x00000000506FEC90>,)

In [58]:
inc_2 = fn(100)

In [59]:
inc_2

<function __main__.incrementer.<locals>.inner.<locals>.inc>

In [60]:
inc_2.__code__.co_freevars

('current', 'n')

In [61]:
inc_2.__closure__

(<cell at 0x0000015F5299B318: int object at 0x00000000506FF8D0>,
 <cell at 0x0000015F5299B798: int object at 0x00000000506FEC90>)

Здесь вы можете видеть, что вторая свободная переменная `n` указывает на ту же ячейку, что и свободная переменная в `fn`.

Обратите внимание, что **a** — это локальная переменная, которая не считается свободной переменной.

И мы можем вызвать замыкания следующим образом:

In [62]:
inc_2()

102

In [63]:
inc_2()

104

In [64]:
inc_3 = incrementer(3)(200)

In [65]:
inc_3()

203

In [66]:
inc_3()

206

---